In [80]:
# ------------------------------
# |
# | Change the list here of the animals you want in the path:
# | Run this chunk 
# |
# ------------------------------
animals = ['cat', 'hippo', 'fox', 'rabbit']

animal_map = {}
index = 1
for animal in animals:
    animal_map.update({animal: index})
    index += 1
print(animal_map)

animal_map = {}
index = 1
for animal in animals:
    animal_map.update({index: animal})
    index += 1

edges1 = []
animal_indices = list(animal_map.keys())

for i in range(len(animal_indices)):
    for j in range(i + 1, len(animal_indices)):
        animal1 = animal_map[animal_indices[i]]
        animal2 = animal_map[animal_indices[j]]
        edges1.append(f"{animal1},{animal2},({animal1}->{animal2})")
        edges1.append(f"{animal2},{animal1},({animal2}->{animal1})")
print(edges1)



{'cat': 1, 'hippo': 2, 'fox': 3, 'rabbit': 4}
['cat,hippo,(cat->hippo)', 'hippo,cat,(hippo->cat)', 'cat,fox,(cat->fox)', 'fox,cat,(fox->cat)', 'cat,rabbit,(cat->rabbit)', 'rabbit,cat,(rabbit->cat)', 'hippo,fox,(hippo->fox)', 'fox,hippo,(fox->hippo)', 'hippo,rabbit,(hippo->rabbit)', 'rabbit,hippo,(rabbit->hippo)', 'fox,rabbit,(fox->rabbit)', 'rabbit,fox,(rabbit->fox)']


In [81]:
import random
from collections import Counter

def parse_graph(edges, animal_map):
    graph = {}
    
    for edge in edges:
        nodes = edge.split(',')
        u, v = nodes[0], nodes[1]
        u_num = animal_map[u]
        v_num = animal_map[v]
        
        if u_num not in graph:
            graph[u_num] = []
        if v_num not in graph:
            graph[v_num] = []
        
        graph[u_num].append(v_num)
        graph[v_num].append(u_num)
    
    return graph

def find_eulerian_cycles(graph, animal_map, num_cycles=50):
    def is_eulerian(graph):
        for node in graph:
            if len(graph[node]) % 2 != 0:
                return False
        return True

    def eulerian_cycle_random(graph):
        start = random.choice(list(graph.keys()))
        stack = [start]
        path = []
        prev_node = None

        while stack:
            u = stack[-1]
            if graph[u]:
                possible_nodes = [v for v in graph[u] if v != prev_node]
                if not possible_nodes:
                    possible_nodes = graph[u]
                v = random.choice(possible_nodes)
                graph[u].remove(v)
                graph[v].remove(u)
                stack.append(v)
                prev_node = u
            else:
                path.append(stack.pop())
        return path[::-1]

    if not is_eulerian(graph):
        return []

    all_cycles = set()
    original_graph = {node: list(neighbors) for node, neighbors in graph.items()}
    reverse_animal_map = {v: k for k, v in animal_map.items()}
    
    while len(all_cycles) < num_cycles:
        graph_copy = {node: list(neighbors) for node, neighbors in original_graph.items()}
        cycle = eulerian_cycle_random(graph_copy)
        cycle_str = ",".join(map(lambda x: reverse_animal_map[x], cycle))
        all_cycles.add(cycle_str)

    return list(all_cycles)

def format_cycles(cycles):
    return "\n".join(cycles)

def check_list(line):
    n = len(line)
    for i in range(n - 2):
        if line[i] == line[i + 2]:
            return False
    return True

def check_eulerian(line):
    return line[0] == line[-1]

def check_animal_counts(line):
    first_animal = line[0]
    animal_counts = Counter(line)
    
    first_animal_okay = animal_counts[first_animal] == len(animals) 
    other_animals_okay = all(count == (len(animals) - 1) for animal, count in animal_counts.items() if animal != first_animal) # 
    
    return first_animal_okay and other_animals_okay

def check_edges_in_line(line, edges):
    for edge in edges:
        animal1, animal2, _ = edge.split(',')
        transition_found = False
        for i in range(len(line) - 1):
            if line[i] == animal1 and line[i + 1] == animal2:
                transition_found = True
                break
        if not transition_found:
            return False
    return True

# ------------------------------
# |
# | Copy + Paste the list of animals above:
# | Copy + Paste the output of the dictionary in animal_map below
# | Copy + Paste the output of the list of edges below
# | Edit num_cycles - stronger computer means faster: for each animal #: 6 animals, 1000 cycles creates 1-3 paths, 7 animals 10000 cycles creates 1-3, and so on 
# |
# ------------------------------

animals = ['cat', 'hippo', 'fox', 'rabbit']
animal_map = {'cat': 1, 'hippo': 2, 'fox': 3, 'rabbit': 4}
edges = ['cat,hippo,(cat->hippo)', 'hippo,cat,(hippo->cat)', 'cat,fox,(cat->fox)', 'fox,cat,(fox->cat)', 'cat,rabbit,(cat->rabbit)', 'rabbit,cat,(rabbit->cat)', 'hippo,fox,(hippo->fox)', 'fox,hippo,(fox->hippo)', 'hippo,rabbit,(hippo->rabbit)', 'rabbit,hippo,(rabbit->hippo)', 'fox,rabbit,(fox->rabbit)', 'rabbit,fox,(rabbit->fox)']
num_cycles = 100


graph = parse_graph(edges, animal_map)
cycles = find_eulerian_cycles(graph, animal_map, num_cycles) 
formatted_cycles = format_cycles(cycles)

counter = 0
add = True
with open('valid_seeds.txt', 'a') as f:
    for line in cycles:
        line = line.strip().split(',')
        
        check1 = check_list(line)
        check2 = check_eulerian(line)
        check3 = check_animal_counts(line)
        check4 = check_edges_in_line(line, edges)
        if counter < 65536:
            add = True
        else: 
            add = False
        if check1 and check2 and check3 and add and check4:
            f.write(",".join(line) + "\n")
            counter += 1
print(counter)

# THIS RUNS IN O(N^2) TIME 
# THIS CODE IS NOT OPTIMIZED, IT TAKES FOREVER WITH MORE THAN 7 ANIMALS 

4
